In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from project_lib import Project
import types
from botocore.client import Config
import ibm_boto3
import ibmos2spark
import types
import pandas as pd




client_29882ce9c0f74130ab5806368b60732b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='fr5UusEaEba2L3BgsHv_p91SAwsntg4rwsdUk8y51MMd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')



body = client_29882ce9c0f74130ab5806368b60732b.get_object(Bucket='trainclassifier-donotdelete-pr-avgynhohutocd8',Key='test_data.csv')['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)

project = Project(sc,"6a9ddb2e-72df-4e1b-bba5-c1764c25c533","p-ccf0395c1a40d8af007b1009245f2892bc8e300c")


col=['summary','resolution']
df=dataset[col]
df['resolution']=df['resolution'].apply(lambda s : s.strip())
df['category_id'] = df['resolution'].factorize()[0]
category_id_df = df[['category_id','resolution']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)


corpus = []
for i in range(0, len(df)):
  review = re.sub('[^a-zA-Z]', ' ', df['summary'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)
  


X_train, X_test, y_train, y_test = train_test_split(df['summary'], df['category_id'], test_size = 0.20, random_state = 21)
tfidf_transformer = TfidfVectorizer()
X_train = tfidf_transformer.fit_transform(list(X_train))



classifier=LinearSVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(tfidf_transformer.transform(X_test.tolist()))
y_test = np.vectorize(category_to_id.get)(y_test)
y_pred = np.vectorize(category_to_id.get)(y_pred)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
#print(cm)
print(accuracy_score(y_test, y_pred))


project.save_data(file_name = "category_id.csv",data = category_id_df.to_csv(index=False),overwrite=True)
project.save_data(file_name = "tfidf.pickle",data = pickle.dumps(tfidf_transformer),overwrite=True)
project.save_data(file_name = "svcclassifier.pickle",data = pickle.dumps(classifier),overwrite=True)






Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200906070151-0010
KERNEL_ID = 8ccbdb0c-a68e-402e-950e-b7ad44a8e4d1


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/spark/shared/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.5555555555555556


{'file_name': 'svcclassifier.pickle',
 'message': 'File saved to project storage.',
 'bucket_name': 'trainclassifier-donotdelete-pr-avgynhohutocd8',
 'asset_id': 'eb0151e9-3036-4039-8924-74b3255636ba'}